# import

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
import random
import math
import os
import yaml
from pathlib import Path
from IPython.display import Image, display

# Hydra는 주피터 노트북에서 특별한 초기화 방식이 필요합니다.
import hydra
from hydra import initialize, compose


from modellib import build, loader, objective
from modellib.utils import plots, details, history, gcam

## setting cofig load and path

In [ ]:
# 1. 설정 파일 로드
initialize(config_path="conf", version_base=None) 
cfg = compose(config_name="config")

# --- 기본 설정 적용 ---
# 재현성을 위한 시드 고정
torch.manual_seed(cfg.seed)
np.random.seed(cfg.seed)
random.seed(cfg.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(cfg.seed)

# GPU 설정
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- 결과 저장 경로 설정 ---
# 실험 이름으로 최상위 결과 폴더 생성
output_dir = Path(f"results/{cfg.experiment_name}")
hpo_dir = output_dir / cfg.results_subdirs.hpo
models_dir = output_dir / cfg.results_subdirs.models
metrics_dir = output_dir / cfg.results_subdirs.metrics
gradcam_dir = output_dir / cfg.results_subdirs.gradcam

# 폴더 생성
hpo_dir.mkdir(parents=True, exist_ok=True)
models_dir.mkdir(parents=True, exist_ok=True)
metrics_dir.mkdir(parents=True, exist_ok=True)
gradcam_dir.mkdir(parents=True, exist_ok=True)

print(f"🎉 설정 로드 완료! 실험 '{cfg.experiment_name}'을(를) 시작합니다.")
print(f"DEVICE: {DEVICE}")
print(f"결과는 '{output_dir}' 폴더에 저장됩니다.") in: {RUNS_DIR.resolve()}")

# optuna

In [ ]:
import optuna

opt_hp_path = hpo_dir / "opt_hp.yaml"

if cfg.hpo.enabled:
    print("\n--- 🚀 1단계: 하이퍼파라미터 최적화(HPO) 시작 ---")
    
    # Objective 함수 인스턴스 생성
    objective_func = objective.Objective(
        config=cfg, # config 전체를 넘겨 search_space를 사용하도록 함
        data_dir=cfg.data_dir,
        backbone=cfg.backbone,
        max_epochs=cfg.hpo.max_epochs_per_trial,
        n_splits=cfg.split_config.n_splits_cv,
        metric_to_optimize=cfg.hpo.metric_to_optimize,
        test_ratio=cfg.split_config.test_ratio
    )

    # Optuna Study 생성
    study = optuna.create_study(
        study_name=cfg.experiment_name,
        direction=objective_func.direction,
        sampler=optuna.samplers.TPESampler(seed=cfg.seed),
        pruner=optuna.pruners.HyperbandPruner()
    )

    # HPO 실행
    study.optimize(objective_func, n_trials=cfg.hpo.n_trials)

    # 최적 하이퍼파라미터 저장
    best_params = study.best_trial.params
    with open(opt_hp_path, 'w') as f:
        yaml.dump(best_params, f, default_flow_style=False)
        
    print(f"\n✅ HPO 완료! 최적 하이퍼파라미터를 '{opt_hp_path}'에 저장했습니다.")
    print("최적 하이퍼파라미터:", best_params)

    best_trial = study.best_trial
    if 'target_steps_median' not in best_trial.user_attrs:
        raise RuntimeError("best_trial.user_attrs에 'target_steps_median'이 없습니다. Objective 코드 수정이 누락된 것 같습니다.")

    target_steps_median = int(best_trial.user_attrs['target_steps_median'])
    print(f"📌 median target steps from CV(best trial): {target_steps_median}")
    
else:
    print("\n--- ⏩ 1단계: HPO 비활성화. 기존 최적 하이퍼파라미터 파일을 로드합니다. ---")
    if not opt_hp_path.exists():
        raise FileNotFoundError(f"HPO가 비활성화되었지만 '{opt_hp_path}' 파일을 찾을 수 없습니다.")
    print(f"'{opt_hp_path}' 파일을 사용합니다.")

# final model train

In [ ]:
print("\n--- 🚀 2단계: 최종 모델 학습 시작 ---")
# 1. 최적 하이퍼파라미터 로드
with open(opt_hp_path, 'r') as f:
    best_hp = yaml.safe_load(f)


In [ ]:
# 2. 최종 학습용 데이터 로더 준비
# CV 때 사용한 Train/Val 데이터를 모두 합쳐서 최종 학습에 사용
# val_ratio=0으로 설정하면, test_ratio만큼 분리된 나머지를 모두 학습용으로 사용
train_final_loader, _, test_loader, class_names = loader.create_dataloaders(
    data_dir=cfg.data_dir,
    backbone=cfg.backbone,
    batch_size=best_hp['batch_size'],
    val_ratio=0, # Val set 없이 전체를 학습에 사용
    test_ratio=cfg.split_config.test_ratio,
    test_dir=cfg.test_dir
)
print(f"최종 학습 데이터셋: {len(train_final_loader.dataset)}개, 테스트 데이터셋: {len(test_loader.dataset)}개")

# steps_per_epoch_full = 배치 업데이트 수
steps_per_epoch_full = len(train_final_loader)
if steps_per_epoch_full <= 0:
    raise RuntimeError("steps_per_epoch_full이 0 이하입니다. full-train dataloader 구성이 잘못되었습니다.")

epochs_final = int(math.ceil(target_steps_median / steps_per_epoch_full))
print(f"🎯 epochs_final = ceil({target_steps_median} / {steps_per_epoch_full}) = {epochs_final}")

epochs_final_path = hpo_dir / "epochs_final.yaml"
with open(epochs_final_path, "w") as f:
    yaml.dump({"epochs_final": epochs_final,
               "target_steps_median": int(target_steps_median),
               "steps_per_epoch_full": int(steps_per_epoch_full),
               "fold_best_steps": best_trial.user_attrs.get("fold_best_steps", []),
               "fold_best_epochs": best_trial.user_attrs.get("fold_best_epochs", [])}, f)
print(f"📝 최종 학습용 epoch 정보를 '{epochs_final_path}'에 저장했습니다.")


In [ ]:
# 2) epochs_final 로드
with open(hpo_dir / "epochs_final.yaml") as f:
    ef = yaml.safe_load(f)
epochs_final = int(ef["epochs_final"])

In [ ]:
# 3. 모델, 옵티마이저, 스케줄러 준비
model = build(
    backbone=cfg.backbone,
    num_classes=len(class_names),
    dropout_rate=best_hp['dropout_rate']
).to(DEVICE)

optimizer = getattr(optim, best_hp['optimizer'])(
    model.parameters(), lr=best_hp['lr'], weight_decay=best_hp.get('weight_decay', 0.0)
)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-6)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs_final):
    # 최종 학습에서는 Train loss만 계산 (Val set이 없으므로)
    # 실제로는 validate_one_epoch을 test_loader로 돌려서 성능을 모니터링 할 수 있음
    train_loss = objective.train_one_epoch(model, train_final_loader, criterion, optimizer, DEVICE)
    val_loss, macro_f1 = objective.validate_one_epoch(model, test_loader, criterion, DEVICE) # 테스트셋으로 검증
    
    print(f"Epoch {epoch+1}/{epochs_final} | Train Loss: {train_loss:.4f} | Test Loss: {val_loss:.4f} | Test Macro-F1: {macro_f1:.4f}")
    
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['macro_f1'].append(macro_f1)
    
    scheduler.step()


print(f"\n✅ 최종 학습 완료! 가장 성능이 좋은 모델을 '{models_dir}'에 저장했습니다.")

In [ ]:
print("\n--- 📋 Saving Model Details ---")

# 모델 상세 정보(파라미터 수, 연산량 등) 저장
details.save_model_details(
    model=model,
    save_dir=metrics_dir,
    model_name=cfg.experiment_name,
    input_size=(1, 3, 224, 224) # 모델에 맞는 사이즈로 조절
)

In [ ]:
#  히스토리 그래프 저장 및 확인
history_plot_path = metrics_dir / f"{cfg.experiment_name}_history.png"
history.save_history_plot(
    history=history,
    save_dir=metrics_dir,
    model_name=cfg.experiment_name
)
display.Image(filename=history_plot_path)

# evaluation

In [ ]:
print("\n--- 📊 Evaluating Final Model on Test Set ---")

# 평가 결과물(리포트, 행렬, 곡선) 저장
plots.save_classification_results(
    model=model,
    dataloader=test_loader,
    class_names=class_names,
    save_dir=metrics_dir,
    model_name=cfg.experiment_name,
    device=DEVICE
)

In [ ]:
print("\n--- 👁️‍🗨️ Generating Grad-CAM Visualizations ---")

gcam.visualize_and_save_gcam_results(
    model=model,
    dataloader=test_loader,
    class_names=class_names,
    save_dir=gradcam_dir,
    device=DEVICE,
    n_true_preds=5,
    n_misclassified=5
)

print("\n🎉 Grad-CAM 분석이 완료되었습니다!")

In [ ]:
print("\n🎉🎉🎉 All processes are complete! 🎉🎉🎉")